In [40]:
import pandas as pd
import numpy as np  
import re
from collections import Counter

In [3]:
df_train = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/train.csv')
df_test = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/test.csv')

In [4]:
# df_train과 df_test를 하나의 데이터프레임으로 합치기
df_combined = pd.concat([df_train, df_test], ignore_index=True)
df_combined

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."
...,...,...,...,...
78009,TEST_23400,NaN,코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속,"코로나19,감염,경로,조사,32.4%,최고,일상감염,지속,기준,확진자,기준,코로나1..."
78010,TEST_23401,NaN,“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란,"여행,외식,자제,vs,모임,시민들,혼란,인천국제공항,아시아나항공,한반도,일주,비행,..."
78011,TEST_23402,NaN,송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹,"임야,송철호,울산,시장,배우자,용인,매입,의혹,송철호,울산,시장,배우자,경기,용인,..."
78012,TEST_23403,NaN,여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사,"여직원,자궁,모형,사진,홍보용,한의사,한의원,간호조무사,동의,자궁,모형,사진,한의사..."


불용어 처리

In [8]:
from collections import Counter

def extract_single_character_keywords(df, keyword_column):
    """
    데이터프레임의 각 행에서 한 글자인 키워드를 추출합니다.
    """
    single_char_keywords = []
    
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            single_char_keywords.extend([keyword for keyword in keywords_list if len(keyword) == 1])
    
    return single_char_keywords

def count_single_character_keywords(df, keyword_column):
    """
    데이터프레임의 각 행에서 한 글자인 키워드를 추출하고, 빈도를 계산하여 딕셔너리 형태로 반환합니다.
    """
    single_char_keywords = extract_single_character_keywords(df, keyword_column)
    keyword_counts = Counter(single_char_keywords)
    return dict(keyword_counts)

single_char_keyword_counts = count_single_character_keywords(df_combined, '키워드')
print(single_char_keyword_counts)

{'6': 1266, '1': 2973, '3': 920, '2': 1469, '초': 8, '4': 706, '8': 250, '5': 480, 'D': 70, '7': 273, '중': 35, '고': 1, 'f': 5, 'B': 76, 'P': 5, 'R': 4, 'L': 1, 'A': 98, '9': 200, '농': 4, '0': 39, '민': 13, '폭': 3, 'G': 8, 'U': 10, 'N': 2, '군': 2, '시': 2, '게': 8, 'M': 6, 'I': 7, '등': 18, 'm': 10, 'e': 9, '남': 6, 'C': 19, '동': 4, 'K': 1, '여': 1, 'Y': 3, 'J': 2, 'H': 19, '콩': 1, 't': 2, '재': 2, '가': 2, 'F': 6, '인': 1, '존': 4, 'z': 1, '면': 1, '관': 1, 'E': 15, 'S': 7, '분': 1, 'X': 6, '통': 4, '착': 1, '탈': 1, '입': 1, 'O': 1, '소': 1, '신': 3, '위': 1, 'W': 1, '거': 1, 'h': 1, '장': 1, '전': 1}


In [9]:
# 주어진 딕셔너리
single_char_keyword_counts = {
    '6': 1266, '1': 2973, '3': 920, '2': 1469, '초': 8, '4': 706, '8': 250, '5': 480, 'D': 70, '7': 273, '중': 35, '고': 1, 
    'f': 5, 'B': 76, 'P': 5, 'R': 4, 'L': 1, 'A': 98, '9': 200, '농': 4, '0': 39, '민': 13, '폭': 3, 'G': 8, 'U': 10, 'N': 2, 
    '군': 2, '시': 2, '게': 8, 'M': 6, 'I': 7, '등': 18, 'm': 10, 'e': 9, '남': 6, 'C': 19, '동': 4, 'K': 1, '여': 1, 'Y': 3, 
    'J': 2, 'H': 19, '콩': 1, 't': 2, '재': 2, '가': 2, 'F': 6, '인': 1, '존': 4, 'z': 1, '면': 1, '관': 1, 'E': 15, 'S': 7, 
    '분': 1, 'X': 6, '통': 4, '착': 1, '탈': 1, '입': 1, 'O': 1, '소': 1, '신': 3, '위': 1, 'W': 1, '거': 1, 'h': 1, '장': 1, 
    '전': 1
}

# 키 값만 추출
keys = single_char_keyword_counts.keys()

# 키 값 출력
print(list(keys))

['6', '1', '3', '2', '초', '4', '8', '5', 'D', '7', '중', '고', 'f', 'B', 'P', 'R', 'L', 'A', '9', '농', '0', '민', '폭', 'G', 'U', 'N', '군', '시', '게', 'M', 'I', '등', 'm', 'e', '남', 'C', '동', 'K', '여', 'Y', 'J', 'H', '콩', 't', '재', '가', 'F', '인', '존', 'z', '면', '관', 'E', 'S', '분', 'X', '통', '착', '탈', '입', 'O', '소', '신', '위', 'W', '거', 'h', '장', '전']


In [29]:
# 한 단어 불용어 리스트
stopwords = ['6', '1', '3', '2', '초', '4', '8', '5', 'D', '7', '중', '고', 
             'f', 'B', 'P', 'R', 'L', 'A', '9', '농', '0', '민', '폭', 'G', 
             'U', 'N', '군', '시', '게', 'M', 'I', '등', 'm', 'e', '남', 'C', 
             '동', 'K', '여', 'Y', 'J', 'H', '콩', 't', '재', '가', 'F', '인', 
             '존', 'z', '면', '관', 'E', 'S', '분', 'X', '통', '착', '탈', '입', 
             'O', '소', '신', '위', 'W', '거', 'h', '장', '전']

def remove_stopwords(keywords_list, stopwords):
    """
    주어진 키워드 리스트에서 불용어를 제거합니다.
    """
    return [keyword for keyword in keywords_list if keyword not in stopwords]

def process_row_keywords(row_keywords, stopwords):
    """
    주어진 행의 키워드 문자열을 분리하고 불용어를 제거합니다.
    """
    keywords_list = row_keywords.split(',')
    return remove_stopwords(keywords_list, stopwords)

def remove_stopwords_from_dataframe(df, keyword_column):
    """
    데이터프레임의 각 행마다 키워드 컬럼에서 불용어를 제거합니다.
    """
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            processed_keywords = process_row_keywords(row[keyword_column], stopwords)
            df.at[index, keyword_column] = ','.join(processed_keywords)
    return df

In [30]:
df = remove_stopwords_from_dataframe(df_combined, '키워드')
print(df_combined)

                ID        분류                                      제목  \
0      TRAIN_00000  문화:전시_공연   용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역            용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역         용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역         수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제       용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                     ...   
78009   TEST_23400       NaN     코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속   
78010   TEST_23401       NaN  “여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란   
78011   TEST_23402       NaN            송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹   
78012   TEST_23403       NaN    여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사   
78013   TEST_23404       NaN                  용인시, 팜 앤 포레스트 조성 '급물살'   

                                                     키워드  
0      용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...  
1      용인,농촌,테마파크

In [32]:
# 불용어 제거 2
def remove_mixed_language_keywords(df, keyword_column):
    """
    데이터프레임의 각 행에서 숫자, 한글, 영어가 혼합된 단어를 제거합니다.
    """
    pattern = re.compile(r'[A-Za-z]+[가-힣]+|[가-힣]+[A-Za-z]+|[0-9]+[가-힣]+|[가-힣]+[0-9]+')
    
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            filtered_keywords = [keyword for keyword in keywords_list if not pattern.search(keyword)]
            df.at[index, keyword_column] = ','.join(filtered_keywords)
    
    return df

# 사용 예시
d_f = pd.DataFrame({'키워드': ['사과,바나나,A씨,11명', '오렌지,포도,12apple,3사람', '딸기,키위,abc123,한글영어123']})
d_f = remove_mixed_language_keywords(d_f, '키워드')
print(d_f)

              키워드
0          사과,바나나
1  오렌지,포도,12apple
2    딸기,키위,abc123


In [27]:
d_f

,키워드
0,"사과,바나나"
1,"오렌지,포도,12apple"
2,"딸기,키위,abc123"


In [33]:
filtered_df = remove_mixed_language_keywords(df_combined, '키워드')
print(filtered_df)

                ID        분류                                      제목  \
0      TRAIN_00000  문화:전시_공연   용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역            용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역         용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역         수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제       용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                     ...   
78009   TEST_23400       NaN     코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속   
78010   TEST_23401       NaN  “여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란   
78011   TEST_23402       NaN            송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹   
78012   TEST_23403       NaN    여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사   
78013   TEST_23404       NaN                  용인시, 팜 앤 포레스트 조성 '급물살'   

                                                     키워드  
0      용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...  
1      용인,농촌,테마파크

In [34]:
# 불용어 제거 3
def remove_specific_keywords(df, keyword_column, keywords_to_remove):
    """
    데이터프레임의 각 행에서 특정 키워드를 제거합니다.
    """
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            filtered_keywords = [keyword for keyword in keywords_list if keyword not in keywords_to_remove]
            df.at[index, keyword_column] = ','.join(filtered_keywords)
    
    return df


keywords_to_remove = ['용인시', '용인']

                    키워드
0            사과,바나나,11명
1            오렌지,포도,3사람
2  딸기,키위,abc123,한글영어123


In [36]:
filtered_df = remove_specific_keywords(filtered_df, '키워드', keywords_to_remove)
print(filtered_df)

                ID        분류                                      제목  \
0      TRAIN_00000  문화:전시_공연   용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역            용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역         용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역         수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제       용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                     ...   
78009   TEST_23400       NaN     코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속   
78010   TEST_23401       NaN  “여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란   
78011   TEST_23402       NaN            송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹   
78012   TEST_23403       NaN    여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사   
78013   TEST_23404       NaN                  용인시, 팜 앤 포레스트 조성 '급물살'   

                                                     키워드  
0      용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...  
1      농촌,테마파크,단체

In [39]:
filtered_df

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","농촌,테마파크,단체,체험객,체험료,지원,체험일,기준,용인시통합예약사이트,접수,용인특..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","노후,주택,에너지,성능,개선,신청,연장,용인시청,노후,건축물,환경친화적,개선,온실,..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,특례시,지정,골..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,권태면,주코스타리카,대사,..."
...,...,...,...,...
78009,TEST_23400,NaN,코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속,"감염,경로,조사,32.4%,최고,일상감염,지속,기준,확진자,기준,신규,집계,전국,확..."
78010,TEST_23401,NaN,“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란,"여행,외식,자제,vs,모임,시민들,혼란,인천국제공항,아시아나항공,한반도,일주,비행,..."
78011,TEST_23402,NaN,송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹,"임야,송철호,울산,시장,배우자,매입,의혹,송철호,울산,시장,배우자,경기,임야,매입,..."
78012,TEST_23403,NaN,여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사,"여직원,자궁,모형,사진,홍보용,한의사,한의원,간호조무사,동의,자궁,모형,사진,한의사..."


각 행에서 키워드 단어로 1번만 쓰인 단어는 전부 제거

In [43]:
def remove_single_occurrence_keywords(df, keyword_column):
    """
    데이터프레임의 각 행에서 1번만 쓰인 단어를 제거합니다.
    """
    # 모든 키워드 결합하여 빈도 계산
    all_keywords = []
    for keywords in df[keyword_column].dropna():
        all_keywords.extend(keywords.split(','))
    
    keyword_counts = Counter(all_keywords)
    
    # 각 행에서 빈도가 1인 단어 제거
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            filtered_keywords = [keyword for keyword in keywords_list if keyword_counts[keyword] > 1]
            df.at[index, keyword_column] = ','.join(filtered_keywords)
    
    return df

     키워드
0  사과,사과
1       
2  키위,키위


In [44]:
filtered_df = remove_single_occurrence_keywords(filtered_df, '키워드')
print(filtered_df)

                ID        분류                                      제목  \
0      TRAIN_00000  문화:전시_공연   용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역            용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역         용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역         수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제       용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                     ...   
78009   TEST_23400       NaN     코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속   
78010   TEST_23401       NaN  “여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란   
78011   TEST_23402       NaN            송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹   
78012   TEST_23403       NaN    여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사   
78013   TEST_23404       NaN                  용인시, 팜 앤 포레스트 조성 '급물살'   

                                                     키워드  
0      용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...  
1      농촌,테마파크,단체

In [46]:
# 분류 컬럼이 결측치인 행
test_df = df_combined[df_combined['분류'].isna()]

# 분류 컬럼이 결측치가 아닌 행
train_df = df_combined[~df_combined['분류'].isna()]

print("Rows with missing '분류':")
print(test_df)

print("\nRows with non-missing '분류':")
print(train_df)

Rows with missing '분류':
               ID   분류                                                 제목  \
54609  TEST_00000  NaN                                     [부고] 김태수씨 별세 외   
54610  TEST_00001  NaN             신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력   
54611  TEST_00002  NaN  [서경이 만난 사람] 전해철 장관 "재정분권 강화 '지방자치 2.0 시대' 마중물 ...   
54612  TEST_00003  NaN                   용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사   
54613  TEST_00004  NaN                             지자체 벽 터 경기지역 산단 활성화 모색   
...           ...  ...                                                ...   
78009  TEST_23400  NaN                코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속   
78010  TEST_23401  NaN             “여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란   
78011  TEST_23402  NaN                       송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹   
78012  TEST_23403  NaN               여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사   
78013  TEST_23404  NaN                             용인시, 팜 앤 포레스트 조성 '급물살'   

                                                   

In [51]:
test_df.to_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/test_df.csv', index=False)
train_df.to_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/train_df.csv', index=False)